In [ ]:
!pip install --quiet diffusers
!pip install --quiet omegaconf
!pip install --quiet transformers
!pip install --quiet accelerate

In [ ]:
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from diffusers.models import AutoencoderKL
import torch
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split

In [ ]:

# Ruta de la carpeta que contiene las imágenes
carpeta_imagenes = "/images"

# Obtener la lista de nombres de archivos de las imágenes
nombres_imagenes = os.listdir(carpeta_imagenes)

# Dividir la lista de nombres de imágenes en conjuntos de entrenamiento y prueba
nombres_imagenes_train, nombres_imagenes_test = train_test_split(nombres_imagenes, test_size=0.2, random_state=42)

# Ahora tienes dos listas: nombres_imagenes_train y nombres_imagenes_test
# con los nombres de las imágenes para el conjunto de entrenamiento y prueba respectivamente


In [ ]:
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-ema").to(device)

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 512



train= CIFAR10(root='./data', train=True,download=True,transform = transform)
trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size,
                                          shuffle=True)

test= CIFAR10(root='./data', train=False,download=True,transform = transform)
testloader = torch.utils.data.DataLoader(test, batch_size=batch_size,
                                          shuffle=True)

In [ ]:
representacions_latents_train=[]
labels_train = []
for data,label in trainloader:
  with torch.no_grad():
    vector = vae.encode(data.to(device)).latent_dist.mode()
    representacions_latents_train.append(vector.view(vector.size(0),-1))
    labels_train.append(label)